In [10]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

# Load the image-label mapping table
mapping_table_path = "updated_mapping_table.csv"  # Update with the path to your mapping table
mapping_table = pd.read_csv(mapping_table_path)

# Drop rows with NaN values in labels
mapping_table.dropna(subset=['crema', 'color_clarity', 'presentation', 'type_of_cup', 'type_of_coffee', 'served_way'], inplace=True)

# Load and preprocess images
images = []
labels = []

images_dir = "augmented_images_folder"  # Folder containing augmented images

for index, row in mapping_table.iterrows():
    image_name = row['image_name']
    image_path = os.path.join(images_dir, image_name)
    
    # Attempt to read the image
    image = cv2.imread(image_path)
    
    # Check if the image was loaded successfully
    if image is None:
        print(f"Warning: Unable to load image '{image_name}'. Skipping...")
        continue
    
    # Resize the image to match input size of VGG16
    image = cv2.resize(image, (224, 224))
    
    # Append the resized image to the list of images
    images.append(image)
    
    # Append the labels to the list of labels
    labels.append(row.drop('image_name').drop('is_relevant').values)  # Exclude 'image_name' and 'is_relevant' from labels

# Convert lists of images to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Convert labels to one-hot encoded format using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
labels_encoded = mlb.fit_transform(labels)

# Perform cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define lists to store evaluation results
val_losses = []
val_accuracies = []

# Iterate over cross-validation folds
for fold, (train_index, val_index) in enumerate(kf.split(images)):
    print(f"Training fold {fold + 1}/5")
    
    # Split data into train and validation sets
    X_train, X_val = images[train_index], images[val_index]
    y_train, y_val = labels_encoded[train_index], labels_encoded[val_index]
    
    # Load pre-trained VGG16 model without top layers
    base_model = VGG16(weights='imagenet', include_top=False)
    
    # Add custom top layers for classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(y_train.shape[1], activation='sigmoid')(x)  # Sigmoid for multi-label classification
    
    # Create final model
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)
    
    # Evaluate the model on validation data
    val_loss, val_accuracy = model.evaluate(X_val, y_val)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

# Calculate average validation metrics
avg_val_loss = np.mean(val_losses)
avg_val_accuracy = np.mean(val_accuracies)

print(f"Average validation loss: {avg_val_loss}")
print(f"Average validation accuracy: {avg_val_accuracy}")

# Save model weights
model.save_weights("model_weights.h5")